In [151]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from collections import deque
from keras import layers, models, optimizers
import random

In [152]:
class DQN(models.Model):
    def __init__(self, action_space):
        super(DQN, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(64, activation='relu')
        self.output_layer = layers.Dense(action_space, activation='linear')

    def call(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        q_values = self.output_layer(x)
        return q_values

In [159]:
class DQNAgent:
    def __init__(self, action_space):
        self.action_space = action_space 
        self.memory = deque(maxlen=10000)
        self.gamma = 0.9  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = DQN(self.action_space)
        self.target_model = DQN(self.action_space)
        self.target_model.set_weights(self.model.get_weights())
        self.optimizer = optimizers.Adam(learning_rate=0.01)
        self.loss_function = tf.keras.losses.Huber()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > 10000:
            del self.memory[0]

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch_experience = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch_experience:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.target_model.predict(next_state)[0])
            target_f = self.target_model.predict(state)
            target_f[0][action] = target
            with tf.GradientTape() as tape:
                q_values = self.model(state)
                loss = self.loss_function(target_f, q_values)
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

In [160]:
env = gym.make('FrozenLake-v1')
action_space = env.action_space.n
state_space = env.observation_space.n
agent = DQNAgent(action_space)

In [161]:
batch_size = 100  # Decreased batch size
num_episodes = 1000  # Increased number of episodes
max_steps = 100  # Increased maximum steps per episode
update_target_frequency = 50  # Decreased update frequency

In [163]:
for episode in range(num_episodes):

    state = env.reset()[0]  # Extracting the integer value from the tuple
    state = np.array(state).reshape(1, -1).astype(np.float32)

    total_reward = 0

    for step in range(max_steps):
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        reward -= 0.01  # Adjust this penalty as needed
        next_state = np.array(next_state).reshape(1, -1).astype(np.float32)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            break

    print("Episode:", episode, "Total Reward:", total_reward)

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

    if episode % update_target_frequency == 0:
        agent.update_target_model()

    if episode % 100 == 0:
        print("Episode:", episode, "Total Reward:", total_reward)


1/1 [==============================] - 0s 11ms/step
Episode: 14 Total Reward: -0.03
1/1 [==============================] - 0s 11ms/step
Episode: 15 Total Reward: -0.07
1/1 [==============================] - 0s 11ms/step
Episode: 16 Total Reward: -0.20000000000000004
1/1 [==============================] - 0s 11ms/step
Episode: 17 Total Reward: -0.03
1/1 [==============================] - 0s 13ms/step
Episode: 18 Total Reward: -0.08
1/1 [==============================] - 0s 12ms/step
Episode: 19 Total Reward: -0.09
1/1 [==============================] - 0s 13ms/step


In [ ]:
total_reward = 0
for _ in range(100):
    state = env.reset()
    state = np.reshape(state, [1, state_space])
    for _ in range(max_steps):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        state = np.reshape(next_state, [1, state_space])
        total_reward += reward
        if done:
            break

print("Average Reward (100 episodes):", total_reward / 100)